In [1]:
import os 
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
import pandas as pd # type: ignore

In [2]:
from utils import get_dataset, get_model_by_tag, get_prompt_template
from LLMAnnotator import LLMAnnotator

In [3]:
os.environ['OPENAI_API_KEY_CLARIN']=os.getenv("OPENAI_API_KEY_CLARIN")
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

token = os.getenv("OPENAI_API_KEY_CLARIN")

In [4]:
import pandas as pd
import re

def extract_bracketed_word(df, read_col, write_col):
    """
    Wyodrębnia pierwszy wyraz objęty nawiasami [] z kolumny `read_col`
    i zapisuje go do kolumny `write_col`.
    
    :param df: DataFrame wejściowy
    :param read_col: Nazwa kolumny z której odczytywany jest tekst
    :param write_col: Nazwa kolumny do której zapisywany będzie wynik
    :return: DataFrame z nową kolumną
    """
    def extract_last_bracketed(text):
        matches = re.findall(r'\[([^\[\]]+?)\]', str(text))
        return matches[-1] if matches else None
    
    df[write_col] = df[read_col].apply(extract_last_bracketed)
    return df

In [5]:
import pandas as pd

def validate_text_column(path, df_reference):
    # Wczytaj DataFrame z pliku CSV
    df = pd.read_csv(path)
    
    # Pobierz unikalne wartości z kolumny 'text' z df_reference
    reference_texts = set(df_reference['text'].unique())
    
    # Dodaj kolumnę 'is_valid'
    df['was_in_selected_samples'] = df['text'].isin(reference_texts)
    df = extract_bracketed_word(df, 'output', 'extracted_label')
    df.to_csv(path, index=False)
    
    return df

In [6]:
for i in [4]:
    print('')
    print('')
    print('MUMER', i)
    print('')
    print('')
    model_tag = 'llama3'  
    dataset_tag = 'go_emotions'
    prompt = 'cot_random_samples_cohere'
    temp = 0.3
    number_of_experimet = i
    selected_samples =  f'{model_tag}_random42_96'
    
    model = get_model_by_tag(model_tag, token, temp)  
    dataset_for_annotation = get_dataset(dataset_tag=dataset_tag)
    prompt_txt = get_prompt_template(dataset_tag, prompt)
    examples_for_prompt = pd.read_csv(f'./selected_samples/{dataset_tag}/{selected_samples}.csv')

    output_path = f'./results/{prompt}_temp{temp}/{model_tag}/{dataset_tag}_{model_tag}_{prompt}_{selected_samples}_temp{temp}_exp{number_of_experimet}.csv'
    os.makedirs(f'./results/{prompt}_temp{temp}/{model_tag}', exist_ok=True)
    
    
    text_col = "text"
    label_col = "label"
    output_col = "output"
    
    
    annotator = LLMAnnotator(
        model=model,
        dataset=dataset_for_annotation,
        examples_for_prompt=examples_for_prompt,
        prompt_template=prompt_txt,
        column_text=text_col,
        column_label=label_col,
        column_output=output_col
    )
    
    df = annotator.get_results(is_save=True, output_path=output_path)
    annotator.save_results(output_path=output_path)
    df_new = validate_text_column(output_path, examples_for_prompt)





MUMER 4




/home/julita/Desktop/anote-skrpyty 19.04/LLMAnnotator.py:77: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  self.chain = LLMChain(llm=self.model, prompt=self.prompt)


Nr: 0 Predicted label: To analyze the given text, let's follow the steps outlined:

**Step 1: Summarize the main topic of the text**

The main topic of the text appears to be a commentary on a speech or address referred to as the "State of the Union." However, the speaker questions the authenticity or legitimacy of this title.

**Step 2: Identify any key terms or phrases that hint at a specific category**

Key phrases include "he can *call* it," which implies skepticism or doubt regarding the actual nature of the event being referred to as the "State of the Union." The tone suggests disagreement or disbelief rather than outright anger or hatred.

**Step 3: Compare the topic to the definitions of each category**

Given the skeptical tone and the questioning of the legitimacy of the event's title, categories such as anger, fear, or strong emotions seem less fitting. Instead, the text seems to convey a sense of disbelief or skepticism towards the naming or classification of the event.

**

Failed to multipart ingest runs: Connection error caused failure to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. The content length of 591418742 bytes exceeds the maximum size limit of 20971520 bytes. SSLError(MaxRetryError("HTTPSConnectionPool(host='api.smith.langchain.com', port=443): Max retries exceeded with url: /runs/multipart (Caused by SSLError(SSLError(5, '[SYS] unknown error (_ssl.c:2417)')))"))
Content-Length: 591418742
API Key: lsv2_********************************************7etrace=473c9597-5e1e-4ea1-af15-52627f080e63,id=473c9597-5e1e-4ea1-af15-52627f080e63
